In [18]:
import pandas as pd

Read required files to Pandas

In [33]:
df = pd.read_excel('K:\EOIN\Python\Stock\items.xlsx')
df1 = pd.read_excel('K:\EOIN\Python\Stock\itemfactors.xlsx')
df2 = pd.read_excel('K:\EOIN\Python\Stock\osqty.xlsx')
df3 = pd.read_excel('K:\EOIN\Python\Stock\shipments.xlsx')
df4 = pd.read_excel('K:\EOIN\Python\Stock\physstock.xlsx')
df5 = pd.read_excel('K:\EOIN\Python\Stock\depletionsequiv.xlsx')


Take out item codes that relate to non-stock resale items (Point of sale - POS)

In [34]:
df=df.drop(df.index[df['Gen. Prod. Posting Group']=='POS'])

In order to merge files on item number any in object or string format needs to be amended to int64 (integer/number)

In [35]:
df['No.'] = df['No.'].astype('int64')

Take out item codes that relate to non-stock resale items (PALLETS)

In [36]:
df2=df2.drop(df2.index[df2['No.']=='PALLETS'])

In [37]:
df2['No.'] = df2['No.'].astype('int64')

In [38]:
df3 = df3.dropna(subset=['Item No.'])
df3=df3.drop(df3.index[df3['Item No.']=='Grand Total'])

In [39]:
df3['Item No.'] = df3['Item No.'].astype('int64')

In [40]:
df5=df5.drop(df5.index[df5['Gen. Prod. Posting Group']=='POS'])
df5=df5.drop(df5.index[df5['No.']=='Grand Total'])

In [41]:
df5.dtypes

No.                           object
Item - Search Description     object
Gen. Prod. Posting Group      object
Brand Owner                   object
JAN                          float64
FEB                          float64
MAR                          float64
APR                          float64
MAY                          float64
JUN                          float64
JUL                          float64
AUG                          float64
SEP                          float64
OCT                          float64
NOV                          float64
DEC                          float64
dtype: object

In [42]:
df5['No.'] = df5['No.'].astype('int64')

Create a new column 'phys cases' which relates to book stock on hand

In [43]:
df4['phys cases'] = df4['Sum of Qty Base']/df4['Sum of Qty. per Unit of Measure']

Group and sum to give one result/line for each item number

In [45]:
osq=df2.groupby('No.').sum()

In [46]:
ship=df3.groupby('Item No.').sum()

Merge depletions with item factors and get phys cases

In [47]:
phys = df5.merge(df1, on='No.', how='outer')

Use item factors to convert equivalent cases to physical cases

In [48]:
phys['JAN phys'] = phys['JAN'] / phys['Factor']
phys['FEB phys'] = phys['FEB'] / phys['Factor']
phys['MAR phys'] = phys['MAR'] / phys['Factor']
phys['APR phys'] = phys['APR'] / phys['Factor']

In [49]:
df4.dtypes

No.                                  int64
Description                         object
Brandowner                          object
Group                               object
Group Description                   object
Sum of Qty Base                    float64
Sum of Qty. per Unit of Measure      int64
Sum of P20                         float64
Sum of Valuation                   float64
phys cases                         float64
dtype: object

Merge dataframes and amended dataframes

In [51]:
merge = df.merge(df4, on='No.', how='outer')\
            .merge(osq, on='No.', how='outer')\
            .merge(ship, left_on='No.', right_on='Item No.', how='outer')\
            .merge(phys, on='No.', how='outer')

Now just choose columns you require

In [52]:
Finish=merge.loc[: , ('No.', 'Description_x', 'Description_y', 'Brandowner Code_x', 'Brandowner', 'Brandowner Code_y', 'phys cases', 'Outstanding Quantity', 'JAN_y', 'FEB_y', 'MAR_y', 'APR_y', 'No in case', 'cl ' , 'cl in case', 'Factor', 'JAN phys', 'FEB phys', 'MAR phys', 'APR phys')]

Put zero in every blank cell (NaN)

In [53]:
Final=Finish.fillna(0)

Rename column headings to ease reader understanding

In [54]:
Final.rename(columns = {'phys cases':'Book stock (phys cases)', 'Outstanding Quantity':'Stock on order (phys cases)', 'JAN_y':'JAN equiv', 'FEB_y':'FEB equiv', 'MAR_y':'MAR equiv', 'APR_y':'APR equiv'}, inplace = True)

Now export file to excel

In [57]:
Final.to_excel('current book stock.xlsx')